<a href="https://colab.research.google.com/github/1382Lubster/test_project01/blob/master/Popularity_based_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import Packages needed

In [0]:
import numpy as np
import pandas as pd
#-----------------------------------------------Plotting packages
import matplotlib.pyplot as plt
%matplotlib inline
#-------------------------------------------set column width & supress warnings
pd.set_option('max_colwidth', 500)
import json
import gzip
import warnings
warnings.filterwarnings('ignore')
import os
print(os.getcwd())

C:\Users\user\Desktop\data amazon


# 2.Import data files as Pandas dataframes

In [0]:
#Initialise path to import gz file from json fomat to Pandas DataFrame
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [0]:
#Import meta data 
df1 = pd.read_csv('Pets titles ranks.csv')
Desc1=df1.info()
Desc1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110707 entries, 0 to 110706
Data columns (total 3 columns):
asin         110707 non-null object
title        110497 non-null object
salesRank    108454 non-null object
dtypes: object(3)
memory usage: 2.5+ MB


In [0]:
df1.head()

,asin,title,salesRank
0,0615553605,Pet Qwerks Treat Cookbook with Cutters,{'Pet Supplies': 34134}
1,0615583474,It's A Cat's Life,{'Pet Supplies': 452691}
2,0972585419,Feathered Phonics The Easy Way To Teach Your Bird To Speak Volume 1: 96 Words and Phrases,{'Pet Supplies': 21104}
3,0975412809,Notes From The Grooming Table by Melissa Verplank,{'Pet Supplies': 1960}
4,0975412868,Theory of Five Book by Melissa Verplank,{'Pet Supplies': 70355}


In [0]:
#Import reviews data 
df2 = getDF('reviews_Pet_Supplies.json.gz')
Desc2=df2.info()
Desc2

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1235316 entries, 0 to 1235315
Data columns (total 9 columns):
reviewerID        1235316 non-null object
asin              1235316 non-null object
reviewerName      1226133 non-null object
helpful           1235316 non-null object
reviewText        1235316 non-null object
overall           1235316 non-null float64
summary           1235316 non-null object
unixReviewTime    1235316 non-null int64
reviewTime        1235316 non-null object
dtypes: float64(1), int64(1), object(7)
memory usage: 94.2+ MB


In [0]:
df2.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3PG0KS1YE8MR4,0615553605,Big Sky Gal,"[3, 3]","Recipes are easy and the dogs love them. I would buy this book again and again. Only thing is that the recipes don't tell you how many treats they make, but I suppose that's because you could make them all different sizes. Great buy!",5.0,Great little book!,1354838400,"12 7, 2012"
1,A363P047LR5XI6,0615553605,Miguel Giraldo,"[2, 2]","I received the book in the mail yesterday, the treat cutters are pretty nice and it has different sizes for different sized dogs. I have not made any of the treat/food recipes in the book yet but i looked through it and there's a lot of really nice looking treats with easy ingredients and instructions. I noticed one of them includes garlic powder ( i thought garlic was bad for dogs! ) but i just won't use garlic powder on the recipe asking for it. 4/5 stars because i have not cooked any of t...",4.0,Nice little book,1373932800,"07 16, 2013"
2,ABZ8CQXD42H4,0615553605,Ross Lockhart,"[0, 0]",Not so good. Some of the recipes are not what I would feed my dog. Can not recommend this book,1.0,Did not like the book,1386028800,"12 3, 2013"
3,A3J8QW1MV1OP01,0615583474,Beth Light,"[4, 4]","This is a fun, interactive book that incorporates the story of ""Howie, the World's Greatest Cat"" with helpful information concerning cat behavior and care. Like a ""baby book"" for your cat, it provides places to attach photos, record milestones and memories, store medical records, etc., in one handy place. The heavy-duty pages are in a high-quality binder, and there are very nice pocket pages in the back of the book to hold medical records and the like. The colorful illustrations are cute ...",5.0,This is a Precious Book for Precious Cats,1347321600,"09 11, 2012"
4,A3ISA8Z0NB0ILH,0615583474,"George D. Marshall ""george1767""","[3, 3]",,5.0,Video review of Howie and Friends,1358899200,"01 23, 2013"


In [0]:
#select items active within the last 3 months
df2['reviewTime']=pd.to_datetime(df2['reviewTime'])
df2['Year']=df2['reviewTime'].dt.year# Extract year
df2['Month']=df2['reviewTime'].dt.month #Extract month

#Last year of activity
df2['Year'].max()

#Last date of activity
df2['reviewTime'].max()

#Select items within the last 3 months
df2=df2.loc[df2.Year==2014]# data for 2014 only
df2=df2.loc[df2.Month>4]#data for May, June & July
df2.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,Year,Month
34,A3SFDCRCK2YAJB,0975412809,NaN,"[0, 0]","I had a copy years ago, sold due to funds at the time. Now it's less than half the cost it used to be, got this copy so that I can show references to clients, other groomers, etc. Very handy. Every dog groomer should own this book. This is the grooming Bible. Never think you know enough, there is never enough in this industry, and we're only as smart as our resources. No one can remember 100% of this book by heart, and if you can, please go on Jeopardy because that kind of memory is very rar...",5.0,Can't believe I got rid of this book years ago.,1405296000,2014-07-14,2014,7
48,A3R2MNI9Y6LUE1,0975412809,Debbie Pickett,"[0, 0]",This is an excellent book with great details for each breed and great pictures. I would recommend this book for anyone wanting notes on each breed grooming type.,5.0,Super book,1401148800,2014-05-27,2014,5
52,A2YF4Y717WBTQH,0975412809,Dorene Dennis,"[0, 0]",I bought the book because I am interested in going to grooming school. Overall the book is filled with information on all aspects of dog grooming and care.,4.0,Notes From The Grooming Table,1399075200,2014-05-03,2014,5
70,A1PUK9HN0DRYMZ,0975412809,rachelcorbettphoto,"[0, 0]",I am a groomer and this a an excellent book for refreshers. Especially on breed specific cuts. I use this almost daily =),5.0,Excellent book!!,1400630400,2014-05-21,2014,5
74,A136BYKJMN59R3,0975412809,ryan santos,"[0, 0]",this book is only intended for the dogs actual breed haircut. i live in a world where i have to go by what the customer wants so this didnt really do anything for me.,1.0,not for everyday grooming,1403827200,2014-06-27,2014,6


In [0]:
#select features
df2=df2[['asin','reviewerID','overall']]

# 3.Preprocessing

## 3.a.Preprocessing meta data

In [0]:
#Extract Rank from salesRank using delimiter
df1['salesRank']=df1['salesRank'].astype(str)#Explicitly change datatype to string
df1['Rank'] = df1['salesRank'].str.split(':').str[1].str.replace('[^\w\s]','')#Extract Rank
df1['Rank']=df1['Rank'].astype(float)

In [0]:
#Select fields needed
df1a=df1[['asin','Rank','title']]
df1a=df1a.sort_values(by='Rank', ascending=True)
df1a.head()

,asin,Rank,title
11340,B0009X29WK,1.0,"Precious Cat Ultra Premium Clumping Cat Litter, 40 pound bag"
22861,B000W5SLB8,2.0,"Taste of the Wild Dry Dog Food, Hi Prairie Canine Formula with Roasted Bison &amp; Venison, 30-Pound Bag"
59733,B004QBDO0M,3.0,Bayer K9 Advantage II Flea Control Treatment for Cats
103220,B00EI673B6,5.0,"All-absorb Training Pads 100-count, 22-inch By 23-inch"
35230,B001STX13U,6.0,Fancy Feast


## 3.b. Preprocessing reviews data

In [0]:
#Get average rating & number of reviewers for each item
df2=pd.DataFrame(df2.groupby('asin').agg({'overall':['mean'],'reviewerID':['count']}))
df2.head()

,overall,reviewerID
,mean,count
asin,,
0975412809,4.166667,6
0979699967,4.000000,1
0983794812,5.000000,2
1083812550,4.000000,1
1083813210,2.000000,1


In [0]:
#Drop index at level 1
df2.columns = df2.columns.droplevel(1)
df2.head()

,overall,reviewerID
asin,,
0975412809,4.166667,6
0979699967,4.000000,1
0983794812,5.000000,2
1083812550,4.000000,1
1083813210,2.000000,1


In [0]:
#Rename columns
df2=df2.rename(columns={'overall':'Avg_rating','reviewerID':'Count_Users'})
df2.head()

,Avg_rating,Count_Users
asin,,
0975412809,4.166667,6
0979699967,4.000000,1
0983794812,5.000000,2
1083812550,4.000000,1
1083813210,2.000000,1


In [0]:
df2=df2.reset_index()#Reset Index

In [0]:
#Get items with rating over 4
df2=df2.loc[df2.Avg_rating>4].sort_values(by='Count_Users',ascending=False)
#Get items rated by more than 250 users
df2=df2.loc[df2.Count_Users>250]
df2.head()

,asin,Avg_rating,Count_Users
27016,B00CD0H1ZC,4.823723,1214
3382,B0009X29WK,4.498498,333
14183,B0040QOYZ2,4.713846,325
5610,B000OX64P8,4.612676,284
30065,B00FPKNRCS,4.665339,251


## 3.c. Merge dataframes

In [0]:
df=pd.merge(df2,df1a, on='asin', how='inner')#Merge dataframes x1 & df1 to get  Average rating, Count of users & Rank for each item
df.head()

,asin,Avg_rating,Count_Users,Rank,title
0,B00CD0H1ZC,4.823723,1214,6.0,"Deshedding Tool &amp; Pet Grooming Tool For Small, Medium &amp; Large Dogs + Cats, With Short to Long Hair. Dramatically Reduces Shedding In Minutes GUARANTEED!"
1,B0009X29WK,4.498498,333,1.0,"Precious Cat Ultra Premium Clumping Cat Litter, 40 pound bag"
2,B0040QOYZ2,4.713846,325,1662.0,FURminator deShedding Tool for Dogs
3,B000OX64P8,4.612676,284,718.0,Midwest iCrate Single-Door Home Training and Travel Series
4,B00FPKNRCS,4.665339,251,934.0,"Kyjen Slo-Bowl Slow Feeder Slow Feed Interactive Bloat Stop Dog Bowl, Large"


In [0]:
#Explore dataframe
df.shape

(5, 5)

In [0]:
#Get items with rank below 1000
df=df.loc[df.Rank<1000].sort_values(by='Rank', ascending=True)
#Get top 10 items
Recommendations=df.head(10)
Recommendations

,asin,Avg_rating,Count_Users,Rank,title
1,B0009X29WK,4.498498,333,1.0,"Precious Cat Ultra Premium Clumping Cat Litter, 40 pound bag"
0,B00CD0H1ZC,4.823723,1214,6.0,"Deshedding Tool &amp; Pet Grooming Tool For Small, Medium &amp; Large Dogs + Cats, With Short to Long Hair. Dramatically Reduces Shedding In Minutes GUARANTEED!"
3,B000OX64P8,4.612676,284,718.0,Midwest iCrate Single-Door Home Training and Travel Series
4,B00FPKNRCS,4.665339,251,934.0,"Kyjen Slo-Bowl Slow Feeder Slow Feed Interactive Bloat Stop Dog Bowl, Large"


In [0]:
#Get Product titles for Recommendations: 10 items
Recommendations=pd.DataFrame(Recommendations.sort_values(by='Rank', ascending=True).head(10))#Order by rank & select top 10
Recommendations[['asin','title']]

,asin,title
1,B0009X29WK,"Precious Cat Ultra Premium Clumping Cat Litter, 40 pound bag"
0,B00CD0H1ZC,"Deshedding Tool &amp; Pet Grooming Tool For Small, Medium &amp; Large Dogs + Cats, With Short to Long Hair. Dramatically Reduces Shedding In Minutes GUARANTEED!"
3,B000OX64P8,Midwest iCrate Single-Door Home Training and Travel Series
4,B00FPKNRCS,"Kyjen Slo-Bowl Slow Feeder Slow Feed Interactive Bloat Stop Dog Bowl, Large"
